# 1. Load the python packages

In [ ]:
import pkg_resources
import pip
installedPackages = {pkg.key for pkg in pkg_resources.working_set}
required = {'nltk', 'spacy', 'textblob', 'backtrader'}
missing = required - installedPackages
if missing:
    !pip install nltk==3.4
    !pip install textblob==0.15.3
    !pip install -U SpaCy==2.2.0
    !python -m spacy download en_core_web_lg
    !pip install backtrader==1.9.74.123
!pip install tensorflow
!pip install datasets

In [ ]:
#NLP libraries
from textblob import TextBlob
import spacy
import nltk
import warnings
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
import csv
import pandas as pd

#Run the command python -m spacy download en_core_web_sm to download this
#https://spacy.io/models
import en_core_web_lg
nlp = en_core_web_lg.load()

#Libraries for processing the news headlines
from lxml import etree
import json
from io import StringIO
from os import listdir
from os.path import isfile, join
from pandas.tseries.offsets import BDay
from scipy.stats.mstats import winsorize
from copy import copy

# Libraries for Classification for modeling the sentiments
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Keras package for the deep learning model for the sentiment prediction.
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, LSTM, Dropout, Activation
from tensorflow.keras.layers import Embedding

# Load libraries
import statsmodels.api as sm
import seaborn as sns
import pandas as pd
import numpy as np
import datetime
from datetime import date
import matplotlib.pyplot as plt
import yfinance as yf

#Additional Libraries
import json
import zipfile
import os.path
import sys

#Diable the warnings
import warnings
warnings.filterwarnings('ignore')

# 2. Load dataset

In [ ]:
from google.colab import files
uploaded = files.upload()

# load Step2.2_ReturnData.csv # engineer indicators
# load Raw Headline Data.zip # not needed
# load Step3_NewsAndReturnData.csv # compute new sentiment score
# load Step4_DataWithSentimentsResults.csv # not needed

# 3. Compute new sentiment score

## BERT variants

In [ ]:
from google.colab import files
uploaded = files.upload()

# load roberta LLM distilroberta_9k_ep3_8928.pt

In [ ]:
# Define your tickers list
tickers = ['AAPL', 'MSFT', 'AMZN', 'GOOG', 'META', 'WMT', 'JPM', 'TSLA', 'NFLX', 'ADBE']

# Load the data
data_df = pd.read_csv(r'Step4_DataWithSentimentsResults.csv', sep='|')

# Replace 'FB' with 'META' in the 'ticker' column
data_df['ticker'] = data_df['ticker'].replace('FB', 'META')

# Filter the data to include only rows with the specified tickers
data_df = data_df[data_df['ticker'].isin(tickers)]

In [ ]:
data_df.head()

### RoBERTa

In [ ]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification, AdamW, get_scheduler
import torch
from torch.nn.functional import softmax
import datasets
from datasets import load_dataset, Dataset
import random
from torch.utils.data import DataLoader
import torch.nn.functional as F
import matplotlib.pyplot as plt
from tqdm import tqdm

# Specify the file name from which to load the model
modelsavename = "distilroberta_9k_ep3_8928.pt"

# Initialize the same model architecture
model = RobertaForSequenceClassification.from_pretrained("nickmuchi/distilroberta-finetuned-financial-text-classification")

# Load the saved state_dict into the model
with open(modelsavename, "rb") as f:
    model.load_state_dict(torch.load(f))

# Move model to the device (GPU if available)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

print(f"Model loaded from {modelsavename}")

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained("nickmuchi/distilroberta-finetuned-financial-text-classification")

def get_polarity(input_text):
  # Tokenize input and get model output
  inputs = tokenizer(input_text, return_tensors="pt")

  # Get logits (output for classification)
  outputs = model(**inputs)
  logits = outputs.logits

  # Convert logits to probabilities using softmax
  probs = softmax(logits, dim=1)

  # Map classes to polarity values
  polarity_values = torch.tensor([-1.0, 0.0, 1.0])

  # Calculate polarity score as the weighted sum of probabilities
  polarity_score = torch.sum(probs * polarity_values, dim=1).item()

  # Get the predicted class (optional, for reference)
  predicted_class = logits.argmax(dim=1).item()

  return polarity_score

polarity_score = get_polarity(data_df['headline'][0])
print(polarity_score)

In [ ]:
# Apply the get_polarity function to each headline and store in a new column
data_df['sentiment_roberta'] = data_df['headline'].apply(lambda x: get_polarity(x))

# Optional: Show the first few rows with the new column
data_df.head()

In [ ]:
# data_df.to_csv("Step4_DataWithSentimentsResults_roberta.csv", index=False)
data_df.to_csv(r'Step4_DataWithSentimentsResults_roberta.csv', sep='|', index=False)

### DeBERTa

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW, get_scheduler
import torch
from torch.nn.functional import softmax
import datasets
from datasets import load_dataset, Dataset
import random
from torch.utils.data import DataLoader
import torch.nn.functional as F
import matplotlib.pyplot as plt
from tqdm import tqdm

# Specify the file name from which to load the model
modelsavename = "deberta_9k_ep3_9013.pt"

# Initialize the same model architecture
model = AutoModelForSequenceClassification.from_pretrained("mrm8488/deberta-v3-ft-financial-news-sentiment-analysis")

# Load the saved state_dict into the model
with open(modelsavename, "rb") as f:
    model.load_state_dict(torch.load(f))

# Move model to the device (GPU if available)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

tokenizer = AutoTokenizer.from_pretrained("mrm8488/deberta-v3-ft-financial-news-sentiment-analysis")

print(f"Model loaded from {modelsavename}")

In [ ]:
def get_polarity(input_text):
  # Tokenize input and get model output
  inputs = tokenizer(input_text, return_tensors="pt")

  # Get logits (output for classification)
  outputs = model(**inputs)
  logits = outputs.logits

  # Convert logits to probabilities using softmax
  probs = softmax(logits, dim=1)

  # Map classes to polarity values
  polarity_values = torch.tensor([-1.0, 0.0, 1.0])

  # Calculate polarity score as the weighted sum of probabilities
  polarity_score = torch.sum(probs * polarity_values, dim=1).item()

  # Get the predicted class (optional, for reference)
  predicted_class = logits.argmax(dim=1).item()

  return polarity_score

polarity_score = get_polarity(data_df['headline'][0])
print(polarity_score)

In [ ]:
# Apply the get_polarity function to each headline and store in a new column
data_df['sentiment_deberta'] = data_df['headline'].apply(lambda x: get_polarity(x))

# Optional: Show the first few rows with the new column
data_df.head()

In [ ]:
data_df.to_csv(r'Step4_DataWithSentimentsResults_deberta.csv', sep='|', index=False)

## GPT-2

In [ ]:
from google.colab import files
uploaded = files.upload()

# load GPT LLM distilgpt2_9k_ep3_9657.pt (2-class), distilgpt2_c3_9k_ep3_f1_8705.pt (3-class), gpt2_c3_9k_ep2_f1_8785.pt (3-class)

In [ ]:
# Define your tickers list
tickers = ['AAPL', 'MSFT', 'AMZN', 'GOOG', 'META', 'WMT', 'JPM', 'TSLA', 'NFLX', 'ADBE']

# Load the data
data_df = pd.read_csv(r'Step4_DataWithSentimentsResults.csv', sep='|')

# Replace 'FB' with 'META' in the 'ticker' column
data_df['ticker'] = data_df['ticker'].replace('FB', 'META')

# Filter the data to include only rows with the specified tickers
data_df = data_df[data_df['ticker'].isin(tickers)]

data_df.head()

### GPT-2 (distil)

In [ ]:
import torch
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification, AdamW, get_scheduler
from torch.nn.functional import softmax
import datasets
from datasets import load_dataset, Dataset
import random
from torch.utils.data import DataLoader
import torch.nn.functional as F
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd

# # Specify the file name from which to load the model
# modelsavename = "distilgpt2_c3_9k_ep3_f1_8705.pt" # "distilgpt2_9k_ep3_9657.pt"

# # Initialize the same model architecture
# model = GPT2ForSequenceClassification.from_pretrained("ANGKJ1995/distilgpt2-mbib-4096", num_labels=3, ignore_mismatched_sizes=True) # GPT2ForSequenceClassification.from_pretrained("ANGKJ1995/distilgpt2-mbib-4096")

# # Load the saved state_dict into the model
# with open(modelsavename, "rb") as f:
#     model.load_state_dict(torch.load(f))

# # Move model to the device (GPU if available)
# device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
# model.to(device)

# print(f"Model loaded from {modelsavename}")


# Specify the file name from which to load the model
modelsavename = "distilgpt2_c3_9k_ep3_f1_8705.pt"

# Initialize the model with the same architecture and number of labels
model = GPT2ForSequenceClassification.from_pretrained(
    "ANGKJ1995/distilgpt2-mbib-4096", num_labels=3, ignore_mismatched_sizes=True
)

# Load the saved state_dict into the model
model.load_state_dict(torch.load(modelsavename, map_location=device))

# Move model to the device (GPU if available)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

print(f"Model loaded from {modelsavename}")


In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained("ANGKJ1995/distilgpt2-mbib-4096")

def get_polarity(input_text):
  # Tokenize input and get model output
  inputs = tokenizer(input_text, return_tensors="pt")

  # Get logits (output for classification)
  outputs = model(**inputs)
  logits = outputs.logits

  # Convert logits to probabilities using softmax
  probs = softmax(logits, dim=1)

  # Map classes to polarity values
  polarity_values = torch.tensor([0.0, 1.0])

  # Calculate polarity score as the weighted sum of probabilities
  polarity_score = torch.sum(probs * polarity_values, dim=1).item()
  polarity_score = 2*polarity_score - 1

  # Get the predicted class (optional, for reference)
  predicted_class = logits.argmax(dim=1).item()

  return polarity_score

def get_polarity_c3(input_text):
  # Tokenize input and get model output
  inputs = tokenizer(input_text, return_tensors="pt")

  # Get logits (output for classification)
  outputs = model(**inputs)
  logits = outputs.logits

  # Convert logits to probabilities using softmax
  probs = softmax(logits, dim=1)

  # Map classes to polarity values
  polarity_values = torch.tensor([-1.0, 0.0, 1.0])

  # Calculate polarity score as the weighted sum of probabilities
  polarity_score = torch.sum(probs * polarity_values, dim=1).item()

  # Get the predicted class (optional, for reference)
  predicted_class = logits.argmax(dim=1).item()

  return polarity_score

# polarity_score = get_polarity(data_df['headline'][0])
# print(polarity_score)

polarity_score = get_polarity_c3(data_df['headline'][0])
print(polarity_score)

In [ ]:
# # Apply the get_polarity function to each headline and store in a new column
# data_df['sentiment_gpt2'] = data_df['headline'].apply(lambda x: get_polarity(x))

# # Optional: Show the first few rows with the new column
# data_df.head()

# Apply the get_polarity function to each headline and store in a new column
data_df['sentiment_gpt2_c3'] = data_df['headline'].apply(lambda x: get_polarity_c3(x))

# Optional: Show the first few rows with the new column
data_df.head()

In [ ]:
# data_df.to_csv(r'Step4_DataWithSentimentsResults_gpt2.csv', sep='|', index=False)
data_df.to_csv(r'Step4_DataWithSentimentsResults_gpt2_c3.csv', sep='|', index=False)

### GPT-2 (small)

In [ ]:
import torch
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification, AdamW, get_scheduler, GPT2Config
from torch.nn.functional import softmax
import datasets
from datasets import load_dataset, Dataset
import random
from torch.utils.data import DataLoader
import torch.nn.functional as F
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

modelsavename = "gpt2_c3_9k_ep2_f1_8887.pt" # "gpt2_c3_9k_ep2_f1_8785.pt"

configuration = GPT2Config.from_pretrained("Harshad018/trained-gpt2-tweet-analysis")
configuration.pad_token_id = configuration.eos_token_id  # Use eos_token_id as the pad token

tokenizer = GPT2Tokenizer.from_pretrained("Harshad018/trained-gpt2-tweet-analysis")
tokenizer.pad_token = tokenizer.eos_token  # Set pad token to eos token

model = GPT2ForSequenceClassification.from_pretrained(
    "Harshad018/trained-gpt2-tweet-analysis", config=configuration
).to(device)

model.load_state_dict(torch.load("gpt2_c3_9k_ep2_f1_8887.pt")) # "gpt2_c3_9k_ep2_f1_8785.pt"
model.to(device)

print(f"Model loaded from {modelsavename}")

In [ ]:
def get_polarity_c3(input_text):
  # Tokenize input and get model output
  inputs = tokenizer(input_text, return_tensors="pt")

  # Get logits (output for classification)
  outputs = model(**inputs)
  logits = outputs.logits

  # Convert logits to probabilities using softmax
  probs = softmax(logits, dim=1)

  # Map classes to polarity values
  polarity_values = torch.tensor([-1.0, 0.0, 1.0])

  # Calculate polarity score as the weighted sum of probabilities
  polarity_score = torch.sum(probs * polarity_values, dim=1).item()

  # Get the predicted class (optional, for reference)
  predicted_class = logits.argmax(dim=1).item()

  return polarity_score

# polarity_score = get_polarity(data_df['headline'][0])
# print(polarity_score)

polarity_score = get_polarity_c3(data_df['headline'][0])
print(polarity_score)

In [ ]:
# Apply the get_polarity function to each headline and store in a new column
data_df['sentiment_gpt2_c3_8887'] = data_df['headline'].apply(lambda x: get_polarity_c3(x))

# Optional: Show the first few rows with the new column
data_df.head()

In [ ]:
data_df.to_csv(r'Step4_DataWithSentimentsResults_gpt2_c3_8887.csv', sep='|', index=False)

## T5Encoder

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
# Define your tickers list
tickers = ['AAPL', 'MSFT', 'AMZN', 'GOOG', 'META', 'WMT', 'JPM', 'TSLA', 'NFLX', 'ADBE']

# Load the data
data_df = pd.read_csv(r'Step4_DataWithSentimentsResults.csv', sep='|')

# Replace 'FB' with 'META' in the 'ticker' column
data_df['ticker'] = data_df['ticker'].replace('FB', 'META')

# Filter the data to include only rows with the specified tickers
data_df = data_df[data_df['ticker'].isin(tickers)]

data_df.head()

In [ ]:
from transformers import AutoTokenizer, AutoModel, AdamW, get_scheduler
import torch
from torch.nn.functional import softmax
import datasets
from datasets import load_dataset, Dataset
import random
from torch.utils.data import DataLoader
import torch.nn.functional as F
import matplotlib.pyplot as plt
from tqdm import tqdm

# Specify the file name from which to load the model
modelsavename = "t5encoder_9k_ep3_f1_8903.pt"

tokenizer = AutoTokenizer.from_pretrained("varun-v-rao/t5-base-snli")

# Initialize the same model architecture
encoder_model = AutoModel.from_pretrained("varun-v-rao/t5-base-snli")

class T5EncoderForClassification(torch.nn.Module):
    def __init__(self, model_name, num_labels):
        super(T5EncoderForClassification, self).__init__()
        self.encoder = encoder_model.encoder  # Use only the encoder
        self.classifier = torch.nn.Linear(self.encoder.config.d_model, num_labels)  # Classifier layer

    def forward(self, input_ids, attention_mask=None):
        # Forward pass through the encoder (only encoder, no decoder)
        outputs = self.encoder(input_ids=input_ids, attention_mask=attention_mask)
        # Take the hidden state of the [CLS] token (first token in the sequence)
        cls_output = outputs.last_hidden_state[:, 0, :]
        logits = self.classifier(cls_output)  # Pass the CLS token through the classifier
        return logits

model = T5EncoderForClassification(encoder_model, num_labels=3)

# Load the saved state_dict into the model
with open(modelsavename, "rb") as f:
    model.load_state_dict(torch.load(f))

# Move model to the device (GPU if available)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

print(f"Model loaded from {modelsavename}")

In [ ]:
def get_polarity(input_text):
  # Tokenize input and get model output
  inputs = tokenizer(input_text, return_tensors="pt")
  with torch.no_grad():
    outputs = model(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"])

  # Convert logits to probabilities
  probs = softmax(outputs, dim=1)

  # Map classes to polarity values
  polarity_values = torch.tensor([-1.0, 0.0, 1.0], device=device)

  # Calculate polarity score
  polarity_score = torch.sum(probs * polarity_values, dim=1).item()

  return polarity_score

polarity_score = get_polarity(data_df['headline'][0])
print(polarity_score)

In [ ]:
# Apply the get_polarity function to each headline and store in a new column
data_df['sentiment_t5encoder'] = data_df['headline'].apply(lambda x: get_polarity(x))

# Optional: Show the first few rows with the new column
data_df.head()

In [ ]:
data_df.to_csv(r'Step4_DataWithSentimentsResults_t5encoder.csv', sep='|', index=False)

# 4. Build the trading strategy

In [ ]:
from google.colab import files
uploaded = files.upload()

## Setup

In [ ]:
###########################
# Setting up the strategy #
###########################

import backtrader as bt
import backtrader.indicators as btind
import backtrader.analyzers as btanalyzers

class Sentiment(bt.Indicator):
    lines = ('sentiment',)
    plotinfo = dict(
        plotymargin=0.5,
        plothlines=[0],
        plotyticks=[1.0, 0, -1.0])

    def next(self):
        self.sentiment = 0.0
        self.date = self.data.datetime
        date = bt.num2date(self.date[0]).date()
        prev_sentiment = self.sentiment
        if date in date_sentiment:
            self.sentiment = date_sentiment[date]
        self.lines.sentiment[0] = self.sentiment

class SentimentStrat(bt.Strategy):
    params = (
        ('period', 15),
        ('printlog', True),
    )

    def log(self, txt, dt=None, doprint=False):
        ''' Logging function for this strategy'''
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close
        # Keep track of pending orders
        self.order = None
        self.buyprice = None
        self.buycomm = None
        self.sma = bt.indicators.SimpleMovingAverage(
            self.datas[0], period=self.params.period)
        self.date = self.data.datetime
        self.sentiment = None
        Sentiment(self.data)
        self.plotinfo.plot = False
        self.buy_signals = []  # List to store buy points
        self.sell_signals = []  # List to store sell points

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))
                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    ### Main Strat ###
    def next(self):
        date = bt.num2date(self.date[0]).date()
        prev_sentiment = self.sentiment
        if date in date_sentiment:
            self.sentiment = date_sentiment[date]

        # Check if an order is pending. if yes, we cannot send a 2nd one
        if self.order:
            return
        # If not in the market and previous sentiment not none
        if not self.position and prev_sentiment:
            # buy if current close more than sma AND sentiment increased by >= 0.5
            if self.dataclose[0] > self.sma[0] and self.sentiment - prev_sentiment >= 0.5:
                self.log('Previous Sentiment %.2f, New Sentiment %.2f BUY CREATE, %.2f' % (prev_sentiment, self.sentiment, self.dataclose[0]))
                self.order = self.buy()
                self.buy_signals.append((self.data.datetime.date(0), self.data.close[0]))  # Record buy signal

        # Already in the market and previous sentiment not none
        elif prev_sentiment:
            # sell if current close less than sma AND sentiment decreased by >= 0.5
            if self.dataclose[0] < self.sma[0] and self.sentiment - prev_sentiment <= -0.5:
                self.log('Previous Sentiment %.2f, New Sentiment %.2f SELL CREATE, %.2f' % (prev_sentiment, self.sentiment, self.dataclose[0]))
                self.order = self.sell()
                self.sell_signals.append((self.data.datetime.date(0), self.data.close[0]))  # Record sell signal

    def stop(self):
        self.log('(MA Period %2d) Ending Value %.2f' %
                 (self.params.period, self.broker.getvalue()), doprint=True)

#############################################
# Function for running the trading strategy #
#############################################

def run_strategy(ticker, start, end):
    print(ticker)
    ticker_data = yf.Ticker(ticker)
    df_ticker = ticker_data.history(start=start, end=end)

    # Initialize Cerebro and strategy
    cerebro = bt.Cerebro()
    sentiment_strategy = SentimentStrat  # Assuming your strategy is called SentimentStrat
    cerebro.addstrategy(sentiment_strategy)

    # Load data into Cerebro
    data = bt.feeds.PandasData(dataname=df_ticker)
    cerebro.adddata(data)

    # Set initial cash and other parameters
    initial_cash = 100000.0
    cerebro.broker.setcash(initial_cash)
    cerebro.addsizer(bt.sizers.FixedSize, stake=100)
    print(f'Starting Portfolio Value: {initial_cash:.2f}')

    # Run the strategy
    strat_instance = cerebro.run()[0]  # Get the strategy instance

    # Extract the dates and closing prices
    dates = df_ticker.index
    close_prices = df_ticker['Close']

    # Extract buy/sell signals from the strategy instance
    buy_signals = strat_instance.buy_signals
    sell_signals = strat_instance.sell_signals

    # Plot the closing price and buy/sell markers
    plt.figure(figsize=(12, 6))
    plt.plot(dates, close_prices, label=f"{ticker} Closing Price", color="blue")

    # Plot buy and sell signals
    if buy_signals:
        buy_dates, buy_prices = zip(*buy_signals)
        plt.scatter(buy_dates, buy_prices, marker='^', color='green', label='Buy Signal', s=100)
    if sell_signals:
        sell_dates, sell_prices = zip(*sell_signals)
        plt.scatter(sell_dates, sell_prices, marker='v', color='red', label='Sell Signal', s=100)

    # Customize and display the plot
    plt.title(f"{ticker} Strategy Performance with Buy/Sell Signals")
    plt.xlabel("Date")
    plt.ylabel("Price")
    plt.legend()
    plt.show()

    # Display portfolio summary
    final_cash = cerebro.broker.getvalue()
    print(f'Start Portfolio value: {initial_cash:.2f}\nFinal Portfolio Value: {final_cash:.2f}\nProfit: {final_cash - initial_cash:.2f}')

    return float(close_prices[0]), (final_cash - initial_cash)

## RoBERTa

In [ ]:
# Define your tickers list
tickers = ['AAPL', 'MSFT', 'AMZN', 'GOOG', 'META', 'WMT', 'JPM', 'TSLA', 'NFLX', 'ADBE']

# Load the data
data_df = pd.read_csv(r'Step4_DataWithSentimentsResults_roberta.csv', sep='|')

# Replace 'FB' with 'META' in the 'ticker' column
data_df['ticker'] = data_df['ticker'].replace('FB', 'META')

# Filter the data to include only rows with the specified tickers
data_df = data_df[data_df['ticker'].isin(tickers)]

#### Results for Individual Stocks

In [ ]:
ticker = 'GOOG'
date_sentiment=data_df[data_df['ticker'].isin([ticker])]
date_sentiment=date_sentiment[['date','sentiment_roberta']]
date_sentiment['date']=pd.to_datetime(date_sentiment['date'], format='%Y-%m-%d').dt.date
date_sentiment=date_sentiment.set_index('date')['sentiment_roberta']
date_sentiment=date_sentiment.to_dict()
run_strategy(ticker, start = '2012-01-01', end = '2018-12-12')

#### Results for Multiple Stocks

In [ ]:
results_tickers = {}
for ticker in tickers:
    date_sentiment=data_df[data_df['ticker'].isin([ticker])]
    date_sentiment=date_sentiment[['date','sentiment_roberta']]
    date_sentiment['date']=pd.to_datetime(date_sentiment['date'], format='%Y-%m-%d').dt.date
    date_sentiment=date_sentiment.set_index('date')['sentiment_roberta']
    date_sentiment=date_sentiment.to_dict()
    results_tickers[ticker] = run_strategy(ticker, start = '2012-01-01', end = '2018-12-12')

In [ ]:
df_profit_all = pd.DataFrame.from_dict(results_tickers).set_index([pd.Index(["PerUnitStartPrice", 'StrategyProfit'])])

df_profit_all

In [ ]:
strategy_profit_sum = df_profit_all.loc["StrategyProfit"].sum()
print("Total Strategy Profit:", strategy_profit_sum)

#### Varying the Strategy Time Period for Individual Stocks

In [ ]:
import random
import pandas as pd
from datetime import timedelta

# Assuming your data is loaded as 'data_df' and spans '2012-01-01' to '2018-12-12'
ticker = 'GOOG'
start_date = pd.to_datetime('2012-01-01')
end_date = pd.to_datetime('2018-12-12')
total_days = (end_date - start_date).days

# Set seed for reproducibility
random.seed(42)

# Generate 100 random start dates within the allowable range
period_length = 365*2
num_period = 100
periods = [
    start_date + timedelta(days=day_offset)
    for day_offset in random.sample(range(total_days - period_length), num_period)
]

# Calculate and store profit for each period
profits = []
for start in periods:
    period_start = start.strftime('%Y-%m-%d')
    period_end = (start + timedelta(days=period_length)).strftime('%Y-%m-%d')
    _, profit = run_strategy(ticker, start=period_start, end=period_end)
    profits.append((period_start, period_end, profit))

# Convert to a DataFrame for analysis
profit_df = pd.DataFrame(profits, columns=["Period Start", "Period End", "Profit"])
print(profit_df)

In [ ]:
# Display descriptive statistics for the 'Profit' column
profit_stats = profit_df['Profit'].describe()
print(profit_stats)

#### Varying the Strategy Time Period for Multiple Stocks

In [ ]:
import random
import pandas as pd
from datetime import timedelta

# Assuming your data is loaded as 'data_df' and spans '2012-01-01' to '2018-12-12'
tickers = ['AAPL', 'MSFT', 'AMZN', 'GOOG', 'META', 'WMT', 'JPM', 'TSLA', 'NFLX', 'ADBE']
start_date = pd.to_datetime('2012-01-01')
end_date = pd.to_datetime('2018-12-12')
total_days = (end_date - start_date).days

# Set seed for reproducibility
random.seed(42)

# Define period length and number of periods
period_length = 365*2  # Length of each period in days
num_period = 100    # Number of periods

# Dictionary to store descriptive stats for each ticker
descriptive_stats = {}

# Iterate over each ticker
for ticker in tickers:
    # Generate 100 random start dates within the allowable range
    periods = [
        start_date + timedelta(days=day_offset)
        for day_offset in random.sample(range(total_days - period_length), num_period)
    ]

    # Calculate and store profit for each period
    profits = []
    for start in periods:
        period_start = start.strftime('%Y-%m-%d')
        period_end = (start + timedelta(days=period_length)).strftime('%Y-%m-%d')
        _, profit = run_strategy(ticker, start=period_start, end=period_end)
        profits.append(profit)

    # Calculate descriptive statistics for the ticker
    profit_series = pd.Series(profits)
    stats = profit_series.describe()

    # Add ticker stats to dictionary
    descriptive_stats[ticker] = stats

# Convert the dictionary to a DataFrame
descriptive_stats_df = pd.DataFrame(descriptive_stats)

In [ ]:
descriptive_stats_df

## DeBERTa

In [ ]:
# Define your tickers list
tickers = ['AAPL', 'MSFT', 'AMZN', 'GOOG', 'META', 'WMT', 'JPM', 'TSLA', 'NFLX', 'ADBE']

# Load the data
data_df = pd.read_csv(r'Step4_DataWithSentimentsResults_deberta.csv', sep='|')

# Replace 'FB' with 'META' in the 'ticker' column
data_df['ticker'] = data_df['ticker'].replace('FB', 'META')

# Filter the data to include only rows with the specified tickers
data_df = data_df[data_df['ticker'].isin(tickers)]

#### Results for Individual Stocks

In [ ]:
ticker = 'GOOG'
date_sentiment=data_df[data_df['ticker'].isin([ticker])]
date_sentiment=date_sentiment[['date','sentiment_deberta']]
date_sentiment['date']=pd.to_datetime(date_sentiment['date'], format='%Y-%m-%d').dt.date
date_sentiment=date_sentiment.set_index('date')['sentiment_deberta']
date_sentiment=date_sentiment.to_dict()
run_strategy(ticker, start = '2012-01-01', end = '2018-12-12')

#### Results for Multiple Stocks

In [ ]:
results_tickers = {}
for ticker in tickers:
    date_sentiment=data_df[data_df['ticker'].isin([ticker])]
    date_sentiment=date_sentiment[['date','sentiment_deberta']]
    date_sentiment['date']=pd.to_datetime(date_sentiment['date'], format='%Y-%m-%d').dt.date
    date_sentiment=date_sentiment.set_index('date')['sentiment_deberta']
    date_sentiment=date_sentiment.to_dict()
    results_tickers[ticker] = run_strategy(ticker, start = '2012-01-01', end = '2018-12-12')

In [ ]:
df_profit_all = pd.DataFrame.from_dict(results_tickers).set_index([pd.Index(["PerUnitStartPrice", 'StrategyProfit'])])

df_profit_all

In [ ]:
strategy_profit_sum = df_profit_all.loc["StrategyProfit"].sum()
print("Total Strategy Profit:", strategy_profit_sum)

#### Varying the Strategy Time Period for Individual Stocks

In [ ]:
import random
import pandas as pd
from datetime import timedelta

# Assuming your data is loaded as 'data_df' and spans '2012-01-01' to '2018-12-12'
ticker = 'GOOG'
start_date = pd.to_datetime('2012-01-01')
end_date = pd.to_datetime('2018-12-12')
total_days = (end_date - start_date).days

# Set seed for reproducibility
random.seed(42)

# Generate 100 random start dates within the allowable range
period_length = 365*2
num_period = 100
periods = [
    start_date + timedelta(days=day_offset)
    for day_offset in random.sample(range(total_days - period_length), num_period)
]

# Calculate and store profit for each period
profits = []
for start in periods:
    period_start = start.strftime('%Y-%m-%d')
    period_end = (start + timedelta(days=period_length)).strftime('%Y-%m-%d')
    _, profit = run_strategy(ticker, start=period_start, end=period_end)
    profits.append((period_start, period_end, profit))

# Convert to a DataFrame for analysis
profit_df = pd.DataFrame(profits, columns=["Period Start", "Period End", "Profit"])
print(profit_df)

In [ ]:
# Display descriptive statistics for the 'Profit' column
profit_stats = profit_df['Profit'].describe()
print(profit_stats)

#### Varying the Strategy Time Period for Multiple Stocks

In [ ]:
import random
import pandas as pd
from datetime import timedelta

# Assuming your data is loaded as 'data_df' and spans '2012-01-01' to '2018-12-12'
tickers = ['AAPL', 'MSFT', 'AMZN', 'GOOG', 'META', 'WMT', 'JPM', 'TSLA', 'NFLX', 'ADBE']
start_date = pd.to_datetime('2012-01-01')
end_date = pd.to_datetime('2018-12-12')
total_days = (end_date - start_date).days

# Set seed for reproducibility
random.seed(42)

# Define period length and number of periods
period_length = 365*2  # Length of each period in days
num_period = 100    # Number of periods

# Dictionary to store descriptive stats for each ticker
descriptive_stats = {}

# Iterate over each ticker
for ticker in tickers:
    # Generate 100 random start dates within the allowable range
    periods = [
        start_date + timedelta(days=day_offset)
        for day_offset in random.sample(range(total_days - period_length), num_period)
    ]

    # Calculate and store profit for each period
    profits = []
    for start in periods:
        period_start = start.strftime('%Y-%m-%d')
        period_end = (start + timedelta(days=period_length)).strftime('%Y-%m-%d')
        _, profit = run_strategy(ticker, start=period_start, end=period_end)
        profits.append(profit)

    # Calculate descriptive statistics for the ticker
    profit_series = pd.Series(profits)
    stats = profit_series.describe()

    # Add ticker stats to dictionary
    descriptive_stats[ticker] = stats

# Convert the dictionary to a DataFrame
descriptive_stats_df = pd.DataFrame(descriptive_stats)

In [ ]:
descriptive_stats_df

## GPT-2 distil (2-class)

trained on LabelledNewsData.csv

In [ ]:
# Define your tickers list
tickers = ['AAPL', 'MSFT', 'AMZN', 'GOOG', 'META', 'WMT', 'JPM', 'TSLA', 'NFLX', 'ADBE']

# Load the data
data_df = pd.read_csv(r'Step4_DataWithSentimentsResults_gpt2.csv', sep='|')

# Replace 'FB' with 'META' in the 'ticker' column
data_df['ticker'] = data_df['ticker'].replace('FB', 'META')

# Filter the data to include only rows with the specified tickers
data_df = data_df[data_df['ticker'].isin(tickers)]

#### Results for Individual Stocks

In [ ]:
ticker = 'GOOG'
date_sentiment=data_df[data_df['ticker'].isin([ticker])]
date_sentiment=date_sentiment[['date','sentiment_gpt2']]
date_sentiment['date']=pd.to_datetime(date_sentiment['date'], format='%Y-%m-%d').dt.date
date_sentiment=date_sentiment.set_index('date')['sentiment_gpt2']
date_sentiment=date_sentiment.to_dict()
run_strategy(ticker, start = '2012-01-01', end = '2018-12-12')

#### Results for Multiple Stocks

In [ ]:
results_tickers = {}
for ticker in tickers:
    date_sentiment=data_df[data_df['ticker'].isin([ticker])]
    date_sentiment=date_sentiment[['date','sentiment_gpt2']]
    date_sentiment['date']=pd.to_datetime(date_sentiment['date'], format='%Y-%m-%d').dt.date
    date_sentiment=date_sentiment.set_index('date')['sentiment_gpt2']
    date_sentiment=date_sentiment.to_dict()
    results_tickers[ticker] = run_strategy(ticker, start = '2012-01-01', end = '2018-12-12')

In [ ]:
df_profit_all = pd.DataFrame.from_dict(results_tickers).set_index([pd.Index(["PerUnitStartPrice", 'StrategyProfit'])])

df_profit_all

In [ ]:
strategy_profit_sum = df_profit_all.loc["StrategyProfit"].sum()
print("Total Strategy Profit:", strategy_profit_sum)

#### Varying the Strategy Time Period for Individual Stocks

In [ ]:
import random
import pandas as pd
from datetime import timedelta

# Assuming your data is loaded as 'data_df' and spans '2012-01-01' to '2018-12-12'
ticker = 'GOOG'
start_date = pd.to_datetime('2012-01-01')
end_date = pd.to_datetime('2018-12-12')
total_days = (end_date - start_date).days

# Set seed for reproducibility
random.seed(42)

# Generate 100 random start dates within the allowable range
period_length = 365*2
num_period = 100
periods = [
    start_date + timedelta(days=day_offset)
    for day_offset in random.sample(range(total_days - period_length), num_period)
]

# Calculate and store profit for each period
profits = []
for start in periods:
    period_start = start.strftime('%Y-%m-%d')
    period_end = (start + timedelta(days=period_length)).strftime('%Y-%m-%d')
    _, profit = run_strategy(ticker, start=period_start, end=period_end)
    profits.append((period_start, period_end, profit))

# Convert to a DataFrame for analysis
profit_df = pd.DataFrame(profits, columns=["Period Start", "Period End", "Profit"])
print(profit_df)

In [ ]:
# Display descriptive statistics for the 'Profit' column
profit_stats = profit_df['Profit'].describe()
print(profit_stats)

#### Varying the Strategy Time Period for Multiple Stocks

In [ ]:
import random
import pandas as pd
from datetime import timedelta

# Assuming your data is loaded as 'data_df' and spans '2012-01-01' to '2018-12-12'
tickers = ['AAPL', 'MSFT', 'AMZN', 'GOOG', 'META', 'WMT', 'JPM', 'TSLA', 'NFLX', 'ADBE']
start_date = pd.to_datetime('2012-01-01')
end_date = pd.to_datetime('2018-12-12')
total_days = (end_date - start_date).days

# Set seed for reproducibility
random.seed(42)

# Define period length and number of periods
period_length = 365*2  # Length of each period in days
num_period = 100    # Number of periods

# Dictionary to store descriptive stats for each ticker
descriptive_stats = {}

# Iterate over each ticker
for ticker in tickers:
    # Generate 100 random start dates within the allowable range
    periods = [
        start_date + timedelta(days=day_offset)
        for day_offset in random.sample(range(total_days - period_length), num_period)
    ]

    # Calculate and store profit for each period
    profits = []
    for start in periods:
        period_start = start.strftime('%Y-%m-%d')
        period_end = (start + timedelta(days=period_length)).strftime('%Y-%m-%d')
        _, profit = run_strategy(ticker, start=period_start, end=period_end)
        profits.append(profit)

    # Calculate descriptive statistics for the ticker
    profit_series = pd.Series(profits)
    stats = profit_series.describe()

    # Add ticker stats to dictionary
    descriptive_stats[ticker] = stats

# Convert the dictionary to a DataFrame
descriptive_stats_df = pd.DataFrame(descriptive_stats)

In [ ]:
descriptive_stats_df

## GPT-2 distil (3-class)

trained on the same dataset as ROBERTA

In [ ]:
# Define your tickers list
tickers = ['AAPL', 'MSFT', 'AMZN', 'GOOG', 'META', 'WMT', 'JPM', 'TSLA', 'NFLX', 'ADBE']

# Load the data
data_df = pd.read_csv(r'Step4_DataWithSentimentsResults_gpt2_c3.csv', sep='|')

# Replace 'FB' with 'META' in the 'ticker' column
data_df['ticker'] = data_df['ticker'].replace('FB', 'META')

# Filter the data to include only rows with the specified tickers
data_df = data_df[data_df['ticker'].isin(tickers)]

#### Results for Individual Stocks

In [ ]:
ticker = 'GOOG'
date_sentiment=data_df[data_df['ticker'].isin([ticker])]
date_sentiment=date_sentiment[['date','sentiment_gpt2_c3']]
date_sentiment['date']=pd.to_datetime(date_sentiment['date'], format='%Y-%m-%d').dt.date
date_sentiment=date_sentiment.set_index('date')['sentiment_gpt2_c3']
date_sentiment=date_sentiment.to_dict()
run_strategy(ticker, start = '2012-01-01', end = '2018-12-12')

#### Results for Multiple Stocks

In [ ]:
results_tickers = {}
for ticker in tickers:
    date_sentiment=data_df[data_df['ticker'].isin([ticker])]
    date_sentiment=date_sentiment[['date','sentiment_gpt2_c3']]
    date_sentiment['date']=pd.to_datetime(date_sentiment['date'], format='%Y-%m-%d').dt.date
    date_sentiment=date_sentiment.set_index('date')['sentiment_gpt2_c3']
    date_sentiment=date_sentiment.to_dict()
    results_tickers[ticker] = run_strategy(ticker, start = '2012-01-01', end = '2018-12-12')

In [ ]:
df_profit_all = pd.DataFrame.from_dict(results_tickers).set_index([pd.Index(["PerUnitStartPrice", 'StrategyProfit'])])

df_profit_all

In [ ]:
strategy_profit_sum = df_profit_all.loc["StrategyProfit"].sum()
print("Total Strategy Profit:", strategy_profit_sum)

#### Varying the Strategy Time Period for Individual Stocks

In [ ]:
import random
import pandas as pd
from datetime import timedelta

# Assuming your data is loaded as 'data_df' and spans '2012-01-01' to '2018-12-12'
ticker = 'GOOG'
start_date = pd.to_datetime('2012-01-01')
end_date = pd.to_datetime('2018-12-12')
total_days = (end_date - start_date).days

# Set seed for reproducibility
random.seed(42)

# Generate 100 random start dates within the allowable range
period_length = 365*2
num_period = 100
periods = [
    start_date + timedelta(days=day_offset)
    for day_offset in random.sample(range(total_days - period_length), num_period)
]

# Calculate and store profit for each period
profits = []
for start in periods:
    period_start = start.strftime('%Y-%m-%d')
    period_end = (start + timedelta(days=period_length)).strftime('%Y-%m-%d')
    _, profit = run_strategy(ticker, start=period_start, end=period_end)
    profits.append((period_start, period_end, profit))

# Convert to a DataFrame for analysis
profit_df = pd.DataFrame(profits, columns=["Period Start", "Period End", "Profit"])
print(profit_df)

In [ ]:
# Display descriptive statistics for the 'Profit' column
profit_stats = profit_df['Profit'].describe()
print(profit_stats)

#### Varying the Strategy Time Period for Multiple Stocks

In [ ]:
import random
import pandas as pd
from datetime import timedelta

# Assuming your data is loaded as 'data_df' and spans '2012-01-01' to '2018-12-12'
tickers = ['AAPL', 'MSFT', 'AMZN', 'GOOG', 'META', 'WMT', 'JPM', 'TSLA', 'NFLX', 'ADBE']
start_date = pd.to_datetime('2012-01-01')
end_date = pd.to_datetime('2018-12-12')
total_days = (end_date - start_date).days

# Set seed for reproducibility
random.seed(42)

# Define period length and number of periods
period_length = 365*2  # Length of each period in days
num_period = 100    # Number of periods

# Dictionary to store descriptive stats for each ticker
descriptive_stats = {}

# Iterate over each ticker
for ticker in tickers:
    # Generate 100 random start dates within the allowable range
    periods = [
        start_date + timedelta(days=day_offset)
        for day_offset in random.sample(range(total_days - period_length), num_period)
    ]

    # Calculate and store profit for each period
    profits = []
    for start in periods:
        period_start = start.strftime('%Y-%m-%d')
        period_end = (start + timedelta(days=period_length)).strftime('%Y-%m-%d')
        _, profit = run_strategy(ticker, start=period_start, end=period_end)
        profits.append(profit)

    # Calculate descriptive statistics for the ticker
    profit_series = pd.Series(profits)
    stats = profit_series.describe()

    # Add ticker stats to dictionary
    descriptive_stats[ticker] = stats

# Convert the dictionary to a DataFrame
descriptive_stats_df = pd.DataFrame(descriptive_stats)

In [ ]:
descriptive_stats_df

## GPT-2 small (3-class)

trained on the same dataset as ROBERTA

In [ ]:
# Define your tickers list
tickers = ['AAPL', 'MSFT', 'AMZN', 'GOOG', 'META', 'WMT', 'JPM', 'TSLA', 'NFLX', 'ADBE']

# Load the data
data_df = pd.read_csv(r'Step4_DataWithSentimentsResults_gpt2_c3_8887.csv', sep='|')

# Replace 'FB' with 'META' in the 'ticker' column
data_df['ticker'] = data_df['ticker'].replace('FB', 'META')

# Filter the data to include only rows with the specified tickers
data_df = data_df[data_df['ticker'].isin(tickers)]

#### Results for Individual Stocks

In [ ]:
ticker = 'GOOG'
date_sentiment=data_df[data_df['ticker'].isin([ticker])]
date_sentiment=date_sentiment[['date','sentiment_gpt2_c3_8887']]
date_sentiment['date']=pd.to_datetime(date_sentiment['date'], format='%Y-%m-%d').dt.date
date_sentiment=date_sentiment.set_index('date')['sentiment_gpt2_c3_8887']
date_sentiment=date_sentiment.to_dict()
run_strategy(ticker, start = '2012-01-01', end = '2018-12-12')

#### Results for Multiple Stocks

In [ ]:
results_tickers = {}
for ticker in tickers:
    date_sentiment=data_df[data_df['ticker'].isin([ticker])]
    date_sentiment=date_sentiment[['date','sentiment_gpt2_c3_8887']]
    date_sentiment['date']=pd.to_datetime(date_sentiment['date'], format='%Y-%m-%d').dt.date
    date_sentiment=date_sentiment.set_index('date')['sentiment_gpt2_c3_8887']
    date_sentiment=date_sentiment.to_dict()
    results_tickers[ticker] = run_strategy(ticker, start = '2012-01-01', end = '2018-12-12')

In [ ]:
df_profit_all = pd.DataFrame.from_dict(results_tickers).set_index([pd.Index(["PerUnitStartPrice", 'StrategyProfit'])])

df_profit_all

In [ ]:
strategy_profit_sum = df_profit_all.loc["StrategyProfit"].sum()
print("Total Strategy Profit:", strategy_profit_sum)

#### Varying the Strategy Time Period for Individual Stocks

In [ ]:
import random
import pandas as pd
from datetime import timedelta

# Assuming your data is loaded as 'data_df' and spans '2012-01-01' to '2018-12-12'
ticker = 'GOOG'
start_date = pd.to_datetime('2012-01-01')
end_date = pd.to_datetime('2018-12-12')
total_days = (end_date - start_date).days

# Set seed for reproducibility
random.seed(42)

# Generate 100 random start dates within the allowable range
period_length = 365*2
num_period = 100
periods = [
    start_date + timedelta(days=day_offset)
    for day_offset in random.sample(range(total_days - period_length), num_period)
]

# Calculate and store profit for each period
profits = []
for start in periods:
    period_start = start.strftime('%Y-%m-%d')
    period_end = (start + timedelta(days=period_length)).strftime('%Y-%m-%d')
    _, profit = run_strategy(ticker, start=period_start, end=period_end)
    profits.append((period_start, period_end, profit))

# Convert to a DataFrame for analysis
profit_df = pd.DataFrame(profits, columns=["Period Start", "Period End", "Profit"])
print(profit_df)

In [ ]:
# Display descriptive statistics for the 'Profit' column
profit_stats = profit_df['Profit'].describe()
print(profit_stats)

#### Varying the Strategy Time Period for Multiple Stocks

In [ ]:
import random
import pandas as pd
from datetime import timedelta

# Assuming your data is loaded as 'data_df' and spans '2012-01-01' to '2018-12-12'
tickers = ['AAPL', 'MSFT', 'AMZN', 'GOOG', 'META', 'WMT', 'JPM', 'TSLA', 'NFLX', 'ADBE']
start_date = pd.to_datetime('2012-01-01')
end_date = pd.to_datetime('2018-12-12')
total_days = (end_date - start_date).days

# Set seed for reproducibility
random.seed(42)

# Define period length and number of periods
period_length = 365*2  # Length of each period in days
num_period = 100    # Number of periods

# Dictionary to store descriptive stats for each ticker
descriptive_stats = {}

# Iterate over each ticker
for ticker in tickers:
    # Generate 100 random start dates within the allowable range
    periods = [
        start_date + timedelta(days=day_offset)
        for day_offset in random.sample(range(total_days - period_length), num_period)
    ]

    # Calculate and store profit for each period
    profits = []
    for start in periods:
        period_start = start.strftime('%Y-%m-%d')
        period_end = (start + timedelta(days=period_length)).strftime('%Y-%m-%d')
        _, profit = run_strategy(ticker, start=period_start, end=period_end)
        profits.append(profit)

    # Calculate descriptive statistics for the ticker
    profit_series = pd.Series(profits)
    stats = profit_series.describe()

    # Add ticker stats to dictionary
    descriptive_stats[ticker] = stats

# Convert the dictionary to a DataFrame
descriptive_stats_df = pd.DataFrame(descriptive_stats)

In [ ]:
descriptive_stats_df

## T5Encoder (3-class)

#### Results for Individual Stocks

In [ ]:
ticker = 'GOOG'
date_sentiment=data_df[data_df['ticker'].isin([ticker])]
date_sentiment=date_sentiment[['date','sentiment_t5encoder']]
date_sentiment['date']=pd.to_datetime(date_sentiment['date'], format='%Y-%m-%d').dt.date
date_sentiment=date_sentiment.set_index('date')['sentiment_t5encoder']
date_sentiment=date_sentiment.to_dict()
run_strategy(ticker, start = '2012-01-01', end = '2018-12-12')

#### Results for Multiple Stocks

In [ ]:
results_tickers = {}
for ticker in tickers:
    date_sentiment=data_df[data_df['ticker'].isin([ticker])]
    date_sentiment=date_sentiment[['date','sentiment_t5encoder']]
    date_sentiment['date']=pd.to_datetime(date_sentiment['date'], format='%Y-%m-%d').dt.date
    date_sentiment=date_sentiment.set_index('date')['sentiment_t5encoder']
    date_sentiment=date_sentiment.to_dict()
    results_tickers[ticker] = run_strategy(ticker, start = '2012-01-01', end = '2018-12-12')

In [ ]:
df_profit_all = pd.DataFrame.from_dict(results_tickers).set_index([pd.Index(["PerUnitStartPrice", 'StrategyProfit'])])

df_profit_all

In [ ]:
strategy_profit_sum = df_profit_all.loc["StrategyProfit"].sum()
print("Total Strategy Profit:", strategy_profit_sum)

#### Varying the Strategy Time Period for Individual Stocks

In [ ]:
import random
import pandas as pd
from datetime import timedelta

# Assuming your data is loaded as 'data_df' and spans '2012-01-01' to '2018-12-12'
ticker = 'GOOG'
start_date = pd.to_datetime('2012-01-01')
end_date = pd.to_datetime('2018-12-12')
total_days = (end_date - start_date).days

# Set seed for reproducibility
random.seed(42)

# Generate 100 random start dates within the allowable range
period_length = 365*2
num_period = 100
periods = [
    start_date + timedelta(days=day_offset)
    for day_offset in random.sample(range(total_days - period_length), num_period)
]

# Calculate and store profit for each period
profits = []
for start in periods:
    period_start = start.strftime('%Y-%m-%d')
    period_end = (start + timedelta(days=period_length)).strftime('%Y-%m-%d')
    _, profit = run_strategy(ticker, start=period_start, end=period_end)
    profits.append((period_start, period_end, profit))

# Convert to a DataFrame for analysis
profit_df = pd.DataFrame(profits, columns=["Period Start", "Period End", "Profit"])
print(profit_df)

In [ ]:
# Display descriptive statistics for the 'Profit' column
profit_stats = profit_df['Profit'].describe()
print(profit_stats)

#### Varying the Strategy Time Period for Multiple Stocks

In [ ]:
import random
import pandas as pd
from datetime import timedelta

# Assuming your data is loaded as 'data_df' and spans '2012-01-01' to '2018-12-12'
tickers = ['AAPL', 'MSFT', 'AMZN', 'GOOG', 'META', 'WMT', 'JPM', 'TSLA', 'NFLX', 'ADBE']
start_date = pd.to_datetime('2012-01-01')
end_date = pd.to_datetime('2018-12-12')
total_days = (end_date - start_date).days

# Set seed for reproducibility
random.seed(42)

# Define period length and number of periods
period_length = 365*2  # Length of each period in days
num_period = 100    # Number of periods

# Dictionary to store descriptive stats for each ticker
descriptive_stats = {}

# Iterate over each ticker
for ticker in tickers:
    # Generate 100 random start dates within the allowable range
    periods = [
        start_date + timedelta(days=day_offset)
        for day_offset in random.sample(range(total_days - period_length), num_period)
    ]

    # Calculate and store profit for each period
    profits = []
    for start in periods:
        period_start = start.strftime('%Y-%m-%d')
        period_end = (start + timedelta(days=period_length)).strftime('%Y-%m-%d')
        _, profit = run_strategy(ticker, start=period_start, end=period_end)
        profits.append(profit)

    # Calculate descriptive statistics for the ticker
    profit_series = pd.Series(profits)
    stats = profit_series.describe()

    # Add ticker stats to dictionary
    descriptive_stats[ticker] = stats

# Convert the dictionary to a DataFrame
descriptive_stats_df = pd.DataFrame(descriptive_stats)

In [ ]:
descriptive_stats_df